In [1]:
import tensorflow as tf

In [5]:
# MNIST 데이터 다운
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
# CNN 모델 정의
def build_CNN_classifier(x):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # 1st convolution layer
    # kernel 5 x 5 size 32개
    # 28 x 28 x 1 -> 28 x 28 x 32
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 32], stddev=5e-2))
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
    
    # 1st pooling layer
    # 28 x 28 x 31 -> 14 x 14 x 32
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # 2nd convolution layer
    # kernel 5 x 5 size 64개
    # 14 x 14 x 32 -> 14 x 14 x 64
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64], stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
    
    # 2nd pooling layer
    # 14 x 14 x 64 -> 7 x 7 x 64
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # 완전연결층
    # 7 x 7 x 64 (3136) -> 1024
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[7*7*64, 1024], stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    # 출력층
    # 1024 -> 10
    W_output = tf.Variable(tf.truncated_normal(shape=[1024, 10]))
    b_output = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1, W_output) + b_output
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [7]:
# 인풋, 아웃풋 데이터
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

# cnn 선언
y_pred, logits = build_CNN_classifier(x)

# cross entropy 손실함수, 옵티마이저 정의
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [8]:
# 정확도를 계산하는 연산
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
# 세션을 열어 학습 진행
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch = mnist.train.next_batch(50)
        
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x : batch[0], y : batch[1]})
            print('반복(epoch) : %d, 트레이닝 데이터 정확도 : %f'%(i, train_accuracy))
        
        sess.run([train_step], feed_dict={x : batch[0], y : batch[1]})
        
    print('테스트 데이터 정확도 : %f' % (accuracy.eval(feed_dict={x : mnist.test.images, y : mnist.test.labels})))

반복(epoch) : 0, 트레이닝 데이터 정확도 : 0.120000
반복(epoch) : 100, 트레이닝 데이터 정확도 : 0.700000
반복(epoch) : 200, 트레이닝 데이터 정확도 : 0.920000
반복(epoch) : 300, 트레이닝 데이터 정확도 : 0.840000
반복(epoch) : 400, 트레이닝 데이터 정확도 : 0.900000
반복(epoch) : 500, 트레이닝 데이터 정확도 : 0.880000
반복(epoch) : 600, 트레이닝 데이터 정확도 : 0.960000
반복(epoch) : 700, 트레이닝 데이터 정확도 : 0.940000
반복(epoch) : 800, 트레이닝 데이터 정확도 : 0.940000
반복(epoch) : 900, 트레이닝 데이터 정확도 : 0.920000
반복(epoch) : 1000, 트레이닝 데이터 정확도 : 1.000000
반복(epoch) : 1100, 트레이닝 데이터 정확도 : 1.000000
반복(epoch) : 1200, 트레이닝 데이터 정확도 : 1.000000
반복(epoch) : 1300, 트레이닝 데이터 정확도 : 0.920000
반복(epoch) : 1400, 트레이닝 데이터 정확도 : 0.980000
반복(epoch) : 1500, 트레이닝 데이터 정확도 : 0.960000
반복(epoch) : 1600, 트레이닝 데이터 정확도 : 0.920000
반복(epoch) : 1700, 트레이닝 데이터 정확도 : 0.940000
반복(epoch) : 1800, 트레이닝 데이터 정확도 : 1.000000
반복(epoch) : 1900, 트레이닝 데이터 정확도 : 0.940000
반복(epoch) : 2000, 트레이닝 데이터 정확도 : 0.960000
반복(epoch) : 2100, 트레이닝 데이터 정확도 : 0.980000
반복(epoch) : 2200, 트레이닝 데이터 정확도 : 0.940000
반복(epoch) : 2300, 트레이닝 데이터 정확도 : 0.960000
반복(e

In [48]:
with tf.Session() as sess:
    
#     correct_prediction = tf.equal(tf.argmax([1,1], 1), tf.argmax([1,1], 1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(tf.argmax([1,1,2,3,4,5,6])))

6
